In [39]:
import os
from PIL import Image
import matplotlib.pyplot as plt
from os import listdir
import random
import numpy as np
from matplotlib import image
from natsort import natsorted
class preprocess(object):
    def __init__(self, dataset_location="", batch_size=1,shuffle=False):
        self.location=dataset_location
        #idx=[]
        list=os.listdir(self.location)#stores all files names in that folder in list format
        #for i in range(len(list)):
        #    idx.append(i)
        #self.idx=idx    
        self.batch_size=batch_size
        self.length=len(list)
        self.seed=2
        self.shuffle=shuffle
    def __getitem__(self):
        list1=os.listdir(self.location)
        list1=natsorted(list1)
        if self.shuffle==True:
            filename=[]
            random.seed(self.seed)
            #filename=[]
            bsize=self.batch_size
            #filename=random.choices(list1,weights=None,cum_weights=None,k=int(bsize))
            random.shuffle(list1)
            for i in range(self.batch_size):
                filename.append(list1[i])
        elif self.shuffle==False:
            filename=[]
            for i in range(self.batch_size):
                filename.append(list1[i])
        dict={}
        for i in range(self.batch_size):
            a,b=filename[i].split('.')
            dict[a]=self.location+"/"+filename[i]
        return dict
    def crop(self,id1,id2,id3,id4):
        dict_img=self.__getitem__()
        new_imgs={}
        for i in dict_img:
            arr=image.imread(dict_img[i])
            id1n=(id1[0],arr.shape[0]-id1[1])
            id2n=(id2[0],arr.shape[0]-id2[1])
            id3n=(id3[0],arr.shape[0]-id3[1])
            id4n=(id4[0],arr.shape[0]-id4[1])
            new_imgs[i]=arr[id1n[1]:id4n[1]+1,id1n[0]:id2n[0]+1]
        #for i in new_imgs:#
            #plt.imshow(new_imgs[i])#
            #plt.gca().invert_yaxis()
            #plt.show()#
        return new_imgs
    def blur(self):
        dict_img=self.__getitem__()
        new_dict={}
        for i in dict_img:
            arr=image.imread(dict_img[i])
            type=arr.dtype
            #plt.imshow(arr,cmap='gray')#
            #plt.show()#
            temp_arr=np.zeros(arr.shape,dtype=type)
            for j in range(0,arr.shape[0]):
                for k in range(0,arr.shape[1]):
                    temp_arr.flags.writeable=True
                    if len(arr.shape)==2:
                        if j<arr.shape[0]-1 and k<arr.shape[1]-1 and j>0 and k>0:
                            temp_arr[j,k]=np.median(arr[j-1:j+2,k-1:k+2])
                        else:
                            temp_arr[j,k]=arr[j,k]
                    elif len(arr.shape)==3:
                        if j<arr.shape[0]-1 and k<arr.shape[1]-1 and j>0 and k>0:
                            for ch in range(3):
                                temp_arr[j,k,ch]=np.median(arr[j-1:j+2,k-1:k+2,ch])
                        else:
                            for ch in range(3):
                                temp_arr[j,k,ch]=arr[j,k,ch]
            new_dict[i]=temp_arr
        #for l in new_dict:
            #plt.imshow(new_dict[l])
            #plt.show()
        return new_dict
    def rgb2gray(self):
        dict_img=self.__getitem__()
        new_graydict={}
        for i in dict_img:
            arr=image.imread(dict_img[i])
            shape_arr=arr.shape
            if len(shape_arr)>2 and len(shape_arr)==3:
                r,g,b=arr[:,:,0],arr[:,:,1],arr[:,:,2]
                new_graydict[i]=r*0.2989+g*0.5870+b*0.1140
            else:
                new_graydict[i]=np.ndarray.copy(arr)
        #for i in new_graydict:
            #plt.imshow(new_graydict[i],cmap='gray')
            #plt.show()
        return new_graydict
    def translate(self,tx,ty):
        dict_img=self.__getitem__()
        new_dicttranslate={}
        for i in dict_img:
            arr=image.imread(dict_img[i])
            type=str(arr.dtype)
            arr1=arr[::-1,:]#
            #plt.imshow(arr1)#
            #plt.imshow(arr)
            #plt.gca().invert_yaxis()#
            #plt.show()#
            #arr_translate=np.zeros(arr1.shape)#
            arr_translate=np.zeros(arr.shape,dtype=type)
            arr_translate[ty:,tx:]=arr1[:arr1.shape[0]-ty,:arr1.shape[1]-tx]#
            #arr_translate[ty:,tx:]=arr[:arr.shape[0]-ty,:arr.shape[1]-tx]
            new_dicttranslate[i]=arr_translate[::-1,:]
            
            #plt.imshow(new_dicttranslate[i])
            #plt.gca().invert_yaxis()
            #plt.show()
        return new_dicttranslate
    def edge_detection(self):
        dict_img=self.__getitem__()
        new_dict={}
        horizontal_kernel=np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
        for i in dict_img:
            arr=image.imread(dict_img[i])
            if len(arr.shape)>2 and len(arr.shape)==3:
                r,g,b=arr[:,:,0],arr[:,:,1],arr[:,:,2]
                arr=r*0.2989+g*0.5870+b*0.1140
            arr_padded=np.zeros((arr.shape[0]+2,arr.shape[1]+2))
            arr_padded[1:arr_padded.shape[0]-1,1:arr_padded.shape[1]-1]=arr[:,:]
            #arr_padded=np.zeros((arr.shape[0]+4,arr.shape[1]+4))
            #arr_padded[2:arr_padded.shape[0]-2,2:arr_padded.shape[1]-2]=arr[:,:]
            Gx=np.zeros((arr_padded.shape[0]-2,arr_padded.shape[1]-2))
            Gy=np.zeros((arr_padded.shape[0]-2,arr_padded.shape[1]-2))
            for j in range(0,arr_padded.shape[0]-2):
                for k in range(0,arr_padded.shape[1]-2):
                    Gx[j,k]=np.sum(np.multiply(arr_padded[j:j+3,k:k+3],horizontal_kernel))
                    Gy[j,k]=np.sum(np.multiply(arr_padded[j:j+3,k:k+3],-horizontal_kernel.transpose()))
            k=np.sqrt(np.square(Gx)+np.square(Gy))
            #k*=(255.0/k.max())
            k%=255
            new_dict[i]=k
        #for i in new_dict:
            #plt.imshow(new_dict[i],cmap='gray')
            #plt.show()
        return new_dict
    def rotate(self,theta):
        import math
        img_dict=self.__getitem__()
        new_dict={}
        for k in img_dict:
            arr = image.imread(img_dict[k])
            #plt.imshow(arr)
            #plt.show()
            type=str(arr.dtype)
            cosine = np.cos(np.radians(theta))
            sine = np.sin(np.radians(theta))
            height = arr.shape[0]
            width = arr.shape[1]
            new_height = round(abs(arr.shape[0] * cosine) + abs(arr.shape[1] * sine)) + 1
            new_width = round(abs(arr.shape[1] * cosine) + abs(arr.shape[0] * sine)) + 1
            if len(arr.shape)==2:
                output = np.zeros((int(new_height), int(new_width)),dtype=type)
            elif len(arr.shape)>2 and len(arr.shape)==3:
                output = np.zeros((int(new_height), int(new_width),3),dtype=type)
            original_centre_height = round(((arr.shape[0] + 1) / 2) - 1)
            original_centre_width = round(((arr.shape[1] + 1) / 2) - 1)
            new_centre_height = round(((new_height + 1) / 2) - 1)
            new_centre_width = round(((new_width + 1) / 2) - 1)
            for i in range(height):
                for j in range(width):
                    y = arr.shape[0] - 1 - i - original_centre_height
                    x = arr.shape[1] - 1 - j - original_centre_width
                    new_y = math.floor(-x * sine + y * cosine)
                    new_x = math.floor(x * cosine + y * sine)
                    new_y = new_centre_height - new_y
                    new_x = new_centre_width - new_x
                    if 0 <= new_x < new_width and 0 <= new_y < new_height and new_x >= 0 and new_y >= 0:
                        if len(arr.shape)==2:
                            output[int(new_y), int(new_x)] = arr[i, j]
                        elif len(arr.shape)>2 and len(arr.shape)==3:
                            output[int(new_y), int(new_x),:] = arr[i, j]
            new_dict[k]=output
        #for l in new_dict:
                #plt.imshow(new_dict[l],cmap='gray')
                #plt.show()
        return new_dict

    def resize(self,h,w):
        img_dict=self.__getitem__()
        new_dict={}
        h_new=h
        w_new=w
        for k in img_dict:
            arr = image.imread(img_dict[k])
            type=str(arr.dtype)
            height,width=arr.shape[:2]
            h_ratio=h_new/(height)
            w_ratio=w_new/(width)
            if len(arr.shape)==2:
                req = np.zeros((h_new, w_new),dtype=type)
            elif len(arr.shape)>2 and len(arr.shape)==3:
                req = np.zeros((h_new, w_new,3),dtype=type)
            for i in range(h_new):
                for j in range(w_new):
                    h_req = int(i / h_ratio)
                    w_req = int(j / w_ratio)
                    if len(arr.shape)==2:
                        req[i, j] = arr[h_req, w_req]
                    elif len(arr.shape)>2 and len(arr.shape)==3:
                        req[i, j,:] = arr[h_req, w_req]
            new_dict[k]=req
        #for l in new_dict:
                #plt.imshow(new_dict[l],cmap='gray')
                #plt.show()
        return new_dict
    def rescale(self,s):
        import math
        dict_imgs=self.__getitem__()
        new_dict={}
        for im in dict_imgs:
            arr=image.imread(dict_imgs[im])
            height=round(arr.shape[0]*s)
            width=round(arr.shape[1]*s)
            type=str(arr.dtype)
            img_height, img_width =arr.shape[:2]
            #plt.imshow(arr,cmap = "gray")#
            #plt.show()#
            if len(arr.shape) > 2 and len(arr.shape)==3:
                shape1 = (height,width,3)
            elif len(arr.shape) == 2:
                shape1 = (height,width)
            resized = np.empty(shape1)
            resized = resized.astype(type)
            x_ratio = float(img_width - 1) / (width - 1) if width > 1 else 0
            y_ratio = float(img_height - 1) / (height - 1) if height > 1 else 0
            for i in range(height):
                for j in range(width):

                    x_l, y_l = math.floor(x_ratio * j), math.floor(y_ratio * i)
                    x_h, y_h = math.ceil(x_ratio * j), math.ceil(y_ratio * i)

                    x_weight = (x_ratio * j) - x_l
                    y_weight = (y_ratio * i) - y_l

                    a = arr[y_l, x_l]
                    b = arr[y_l, x_h]
                    c = arr[y_h, x_l]
                    d = arr[y_h, x_h]

                    pixel = a * (1 - x_weight) * (1 - y_weight)+ b * x_weight * (1 - y_weight)+c * y_weight * (1 - x_weight)+d * x_weight * y_weight
                    if len(arr.shape)==2:
                        resized[i,j] = pixel
                    elif len(arr.shape)>2 and len(arr.shape)==3:
                        resized[i,j,:]=pixel
            new_dict[im]=resized
        #for l in new_dict:
            #plt.imshow(new_dict[l],cmap='gray')
            #plt.show()
        return new_dict
    
    def contrast_stretching(self,min_val_r,max_val_r,min_val_s,max_val_s):
        import numpy as np
        from PIL import Image
        #slope1 upto min
        #using only min_val_r, max_val_r
        
        slope1  = min_val_s/min_val_r
        slope2 = (max_val_s-min_val_s)/(max_val_r - min_val_r)
        slope3 = (255 - max_val_s)/(255 - max_val_r)
        s1 = slope1
        s2 = slope2
        s3 = slope3
        #pixel normalization
        img_dict=self.__getitem__()
        new_dict={}
        for item in img_dict:
            img = Image.open(img_dict[item])
            #img.show()
            img = np.asarray(img)
            s = img.shape
            op_img = np.zeros(s)
            if len(s)==3 and s[2]==3:
                #color image
                for i in range(3):
                    for j in range(s[0]):
                        for k in range(s[1]):
                            if img[j,k,i]<=min_val_r:
                                y1 = s1*img[j,k,i]
                                op_img[j,k,i] = y1
                            elif img[j,k,i]>min_val_r and img[j,k,i]<=max_val_r:
                                y2 = min_val_s + s2*(img[j,k,i]-min_val_r)
                                op_img[j,k,i] = y2
                            else:
                                y3 = max_val_s + s3*(img[j,k,i]-max_val_r)
                                op_img[j,k,i] = y3
                        
            else:
                for j in range(s[0]):
                    for k in range(s[1]):
                        if img[j,k]<=min_val_r:
                            y1 = s1*img[j,k]
                            op_img[j,k] = y1
                        elif img[j,k]>min_val_r and img[j,k]<=max_val_r:
                            y2 = min_val_s + s2*(img[j,k]-min_val_r)
                            op_img[j,k] = y2
                        else:
                            y3 = max_val_s + s3*(img[j,k]-max_val_r)
                            op_img[j,k] = y3
            op_img = op_img.astype("uint8")
            new_dict[item] = op_img
        return new_dict
    
    def gamma_transform(self,gamma):
        img_dict=self.__getitem__()
        new_dict={}
        for item in img_dict:
            img = Image.open(img_dict[item])
            img = np.asarray(img)
            img = img/255
            s = img.shape
            op_img = np.zeros(s)
            if len(s)==3 and s[2]==3:
            #color image
                for i in range(3):
                    for j in range(s[0]):
                        for k in range(s[1]):
                            op_img[j,k,i] = img[j,k,i]**gamma
            else:
                for j in range(s[0]):
                    for k in range(s[1]):
                        op_img[j,k] = img[j,k]**gamma
        #gray image
            op_img = op_img*255
            op_img = op_img.astype("uint8")
            new_dict[item] = op_img
        return new_dict
    
    #histogram equilization
    def histogram_equalization(self):
        img_dict=self.__getitem__()
        new_dict={}
        for item in img_dict:
            img = Image.open(img_dict[item])
            img = np.asarray(img)
            sh = img.shape
            pixels = sh[0]*sh[1]
            #print(pixels)
            if len(sh)==3 and sh[2]==3:
                #color image
                #print("color image")
                out_img = np.copy(img)
                freq = np.zeros((3,256))
                for i in range(3):
                    for j in range(256):
                        freq[i,j] = np.sum(img[:,:,i]==j)
                #finding pdf
                freq = freq/pixels
                #finding cdf,multiply by 255, round off
                for i in range(3):
                    for j in range(1,256):
                        freq[i,j] = (freq[i,j]+freq[i,j-1])
                freq = np.round_(freq*255)
                #print(freq)
                for i in range(3):
                    for j in range(sh[0]):
                        for k in range(sh[1]):
                            re = img[j,k,i]
                            out_img[j,k,i] = freq[i,re]

            else:
                #gray image
                out_img = np.copy(img)
                freq = np.zeros((1,256))
                for j in range(256):
                    freq[0,j]=np.sum(img==j)
                #finding pdf
                freq = freq/pixels
                #finding cdf, multiply by 255, round off
                for j in range(1,256):
                        freq[0,j] = (freq[0,j]+freq[0,j-1])
                freq = np.round(freq*255)
                #print(freq)
                for j in range(256):
                    out_img[img==j] = freq[0,j]
            out_img = out_img.astype("uint8")
            new_dict[item] = op_img
        return new_dict
    
        #The data type of input image is a string that specifies the address of the input image
        #Data type of output image is 3d numpy array for RGB images and 2D array for gray 
    
    def save_output(self,dicti,locat):
        loc =""
        for i in dicti:
            img  = Image.fromarray(dicti[i])
            loc = '{}/{}'.format(locat,i)
            loc = loc+".png"
            img.save(loc)
            loc = ""
        return ""
inputarg = preprocess("D:\Projects\Image Processing\INPUT",2,False)
#inputarg2 = preprocess('C:\\Users\HP\OneDrive\Documents\ir_test',2,True)
#print(inputarg.__getitem__())
#print(inputarg2.__getitem__())
#dicti = inputarg.crop((150,200),(350,200),(350,0),(150,0))
out_loc = "D:/Projects/Image Processing/contrast_strtching/"

dicti = inputarg.contrast_stretching(80,160,160,240)
inputarg.save_output(dicti,out_loc)
#print(inputarg2.crop((150,200),(350,200),(350,0),(150,0)))
#print(inputarg.blur())
#print(inputarg2.blur())
#print(inputarg2.rgb2gray())
#print(inputarg.translate(100,100))
#print(inputarg2.translate(100,100))
#print(inputarg2.edge_detection())
#print(inputarg2.rotate(360))
#print(inputarg.rescale(2))
#print(inputarg2.rescale(2))
#print(inputarg.resize(100,100))
#print(inputarg2.resize(100,100))

''